# CitiBike Destination Predictor

This project is a collaboration between [Elena Morais](https://github.com/elenasm7) and [Derrick Lewis](https://github.com/lewi0332)

The project requirements are to find and clean data to be fit to a classification algorithm to make predictions on the outcome. 

We chose to use data from CitiBike of all trips in 2018. This data includes time, start/stop locations, and some basic demographic identifiers. To aid in our perdiction we added daily weather information of New York City. 

- Trip Duration (seconds)
- Start Time and Date
- Stop Time and Date
- Start Station Name
- End Station Name
- Station ID
- Station Lat/Long
- Bike ID
- User Type (Customer = 24-hour pass or 3-day pass user; Subscriber = Annual Member)
- Gender (Zero=unknown; 1=male; 2=female)
- Year of Birth

The goal is to perdict the destination neighborhood of each journey based on the available variables. This is a multi-classification problem with a high number of outcomes. We determined that there are 51 qualifying neighborhoods in New York, which presents a sigificant challenge. 


## Data Collection and Cleaning

First we collected 1 year worth of trip data from Cibibike.com. We learn that there are 17million records in the 2018 data, which is too large for our processing capbabilities. Thus we import each month of data, and randomly sample 10% from each month to create a more manageable, yet representative list. 

In [25]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from citibike_scripts import *
np.random.seed(0)
%matplotlib inline

In [4]:
def all_files_to_df(num,start_file_path, end_file_path):
    '''
    this function takes in length of the list of files (+1) and the beginning and end of the file's names,
    it then reads in all the files and concatinates them into a single dataframe.
    '''
    df_li = []
    for i in range(1,num):
        df_li.append(pd.read_csv(start_file_path+ str(i).zfill(2)+end_file_path).sample(frac=.1,random_state=42))
    return pd.concat(df_li)

In [6]:
citibikes_df = all_files_to_df(13,'../citibike_data/2018', '-citibike-tripdata.csv')

In [7]:
citibikes_df.shape

(1754832, 15)

In [8]:
citibikes_df.to_csv('test_data.csv')

### Null values

There are just 295 NaN values missing the station start and end points in the file of nearly 1.8million. This information is crucial to the prediction and thus can not be substituted. We have decidied to simply remove these rows. 

In [9]:
citibikes_df.isnull().sum()

tripduration                 0
starttime                    0
stoptime                     0
start station id           295
start station name         295
start station latitude       0
start station longitude      0
end station id             295
end station name           295
end station latitude         0
end station longitude        0
bikeid                       0
usertype                     0
birth year                   0
gender                       0
dtype: int64

In [12]:
citibikes_df.dropna(inplace=True) #drops rows with missing data, axis default to 0.
citibikes_df.isnull().sum()

tripduration               0
starttime                  0
stoptime                   0
start station id           0
start station name         0
start station latitude     0
start station longitude    0
end station id             0
end station name           0
end station latitude       0
end station longitude      0
bikeid                     0
usertype                   0
birth year                 0
gender                     0
dtype: int64

### Convert to DateTime 

Start and End times are saved in a format that is not readable by Pandas or our future models. We will convert the columns with this information into a standard date format, then split the relevant information into individual columns to be used as a specific independant variable.

In [13]:
citibikes_df['starttime'] = pd.to_datetime(citibikes_df['starttime'])
citibikes_df['start_month'] = citibikes_df.starttime.dt.month
citibikes_df['start_day_of_week'] = citibikes_df.starttime.dt.dayofweek
citibikes_df['start_hour'] = citibikes_df.starttime.dt.hour
citibikes_df['start_date'] = citibikes_df.starttime.dt.day

citibikes_df['starttime'] = citibikes_df.starttime.dt.date
citibikes_df['starttime'] = pd.to_datetime(citibikes_df['starttime'])


In [14]:
citibikes_df.head(2)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,start_month,start_day_of_week,start_hour,start_date
318178,592,2018-01-22,2018-01-22 08:12:31.2680,478.0,11 Ave & W 41 St,40.760301,-73.998842,505.0,6 Ave & W 33 St,40.749013,-73.988484,33395,Subscriber,1986,1,1,0,8,22
383962,450,2018-01-03,2018-01-03 10:07:56.2390,509.0,9 Ave & W 22 St,40.745497,-74.001971,474.0,5 Ave & E 29 St,40.745168,-73.986831,29115,Subscriber,1981,1,1,2,10,3


### Insert Weather Data Here: 

https://www.weather.gov/okx/centralparkhistorical

Next we connect daily weather information. The hypothesis is that this may aide in predicting a destination as fair weather might increase trips to parks and beaches. 

In [16]:
weather_df = pd.read_csv('../citibike_data/2018_weather.csv')
weather_df['starttime'] = pd.to_datetime(weather_df['date'])
weather_df = weather_df.drop(['max_temp', 'min_temp','departure', 'Hdd', 'cdd', 'new_snow', 'date' ], axis=1)                                               

In [17]:
citibikes_df = pd.merge(citibikes_df, weather_df, on='starttime', how='left')

In [19]:
citibikes_df.head(2)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,...,usertype,birth year,gender,start_month,start_day_of_week,start_hour,start_date,ave_temp,precip,snow_depth
0,592,2018-01-22,2018-01-22 08:12:31.2680,478.0,11 Ave & W 41 St,40.760301,-73.998842,505.0,6 Ave & W 33 St,40.749013,...,Subscriber,1986,1,1,0,8,22,43.5,0.01,0
1,450,2018-01-03,2018-01-03 10:07:56.2390,509.0,9 Ave & W 22 St,40.745497,-74.001971,474.0,5 Ave & E 29 St,40.745168,...,Subscriber,1981,1,1,2,10,3,23.0,0.00,0


### Insert Neighborhood Name

Connect Neighborhood name data to our 'End Station Latitude/Longitude' to use as labels for predictions. In the most widely used context there are 51 neighborhoods in New York City. 

In [ ]:
#set the token and submit it
from mapbox import Geocoder
token = 'your_token_here'
geocoder = Geocoder(access_token=token)

end_neighborhoods = add_neighborhoods(big,'end')
start_neighborhoods = add_neighborhoods(big,'start')

In [20]:
end_neighborhoods = pd.read_csv('../citibike_data/end_neighborhoods.csv')
start_neighborhoods = pd.read_csv('../citibike_data/start_neighborhoods.csv')

In [21]:
end_neighborhoods.drop(['Unnamed: 0'], axis=1,inplace=True)
start_neighborhoods.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
end_neighborhoods.drop(['end station latitude','end station longitude'], axis=1,inplace=True)
start_neighborhoods.drop(['start station latitude','start station longitude'],axis=1,inplace=True)

Merge our stop Neighborhoods to the big DataFrame

In [22]:
citibikes_df = pd.merge(citibikes_df, end_neighborhoods, on=['end station latitude','end station longitude'], how='left')

In [24]:
# citibikes_df.isnull().sum()

An error occured with matching, so lets's determine which neighborhoods are missing

In [ ]:
non_match_ends = pd.DataFrame(list(citibikes_df[citibikes_df.end_nhbr.isnull()]['end station name']))
non_match_ends.rename(columns={0:'end station name'},inplace=True)

In [ ]:
matching = pd.merge(non_match_ends,end_neighborhoods, on='end station name', how='left')

It looks like none of the missing values are in our original end neighborhood DataFrame. So, lets get the latitude and longitudes from the missing values in big and create a dataframe of the set of those.

In [ ]:
missing_stops = pd.DataFrame(list(set(zip(citibikes_df[citibikes_df.end_nhbr.isnull()]['end station latitude'],citibikes_df[citibikes_df.end_nhbr.isnull()]['end station longitude']))))
missing_stops.rename(columns={0:'end station latitude',1:'end station longitude'},inplace=True)
missing_stops['end_nhbr'] = missing_stops.apply(lambda row: get_neighborhood(row,'end'), axis=1)
missing_stops.to_csv('citibike_data/missing_end_neighborhoods.csv')

We merged the start neighborhoods with big, and saw a bunch of null values for start neighborhoods as well. So, we processed the steps from above.

In [ ]:
citibikes_df_2 = pd.merge(citibikes_df, start_neighborhoods, on='start station name', how='left')

In [ ]:
non_match_starts = pd.DataFrame(list(citibikes_df_2[citibikes_df_2.start_nhbr.isnull()]['start station name']))
non_match_starts.rename(columns={0:'start station name'},inplace=True)

In [ ]:
missing_starts = pd.DataFrame(list(set(zip(citibikes_df_2[big_2.start_nhbr.isnull()]['start station latitude'],citibikes_df_2[citibikes_df_2.start_nhbr.isnull()]['start station longitude']))))
missing_starts.rename(columns={0:'start station latitude',1:'start station longitude'},inplace=True)
missing_starts['start_nhbr'] = missing_starts.apply(lambda row: get_neighborhood(row,'start'), axis=1)
missing_starts.to_csv('citibike_data/missing_start_neighborhoods.csv')

In [ ]:
len(list(end_neighborhoods.end_nhbr.value_counts()))
# start_neighborhoods

In [ ]:
len(missing_stops.end_nhbr.value_counts())

In [ ]:
stops_total = pd.concat([end_neighborhoods.drop(['end station name'],axis=1),missing_stops])
stops_total.head()

In [ ]:
citibikes_df.columns

In [ ]:
starts_total = pd.concat([start_neighborhoods,missing_starts])
starts_total.shape

In [ ]:
citibikes_df = pd.merge(citibikes_df, stops_total, on=['end station latitude','end station longitude'], how='left')

In [ ]:
citibikes_df.drop(['end_nhbr_x','end station name_x'],inplace=True,axis=1)

In [ ]:
citibikes_df.dropna(inplace=True)

In [ ]:
citibikes_df_2.dropna(inplace=True)

In [ ]:
missing_stops.to_csv('citibike_data/missing_end_neighborhoods.csv')

### Convert usertype to categories

Our CitiBike data includes a label for each trip to determine if it was made by a rider who is an annual subscriber or someone who has purchased a temporary pass. We assume that there may be destinations more likely chosen by tourist users. 

In [ ]:
citibikes_df.usertype = pd.Categorical(citibikes_df.usertype)
citibikes_df.usertype = citibikes_df.usertype.cat.codes

In [ ]:
citibikes_df_2.usertype = pd.Categorical(citibikes_df_2.usertype)
citibikes_df_2.usertype = citibikes_df_2.usertype.cat.codes

### Exploring Age Data

after individually looking at column distributions, there were a few interestings observations with the age data.
Lets look into this below:

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(10,7))
sns.distplot(citibikes_df['birth year'])
plt.title('Distribution of Reported Ages')
plt.savefig('original_age_dist.png')

The easit thing to handle: people born way too early. We've set a cut-off age of 80. Only __0.15%__ is from people saying they were born before 1939. If you look at the data above, these all seem pretty unrealistic, so lets __drop these values.__

In [ ]:
citibikes_df[citibikes_df['birth year'] <= 1939].shape

In [ ]:
round(citibikes_df[citibikes_df['birth year'] <= 1939].shape[0]/big.shape[0] * 100,2)

In [ ]:
citibikes_df.drop(list(citibikes_df[citibikes_df['birth year'] <= 1939]['birth year'].index),inplace=True)

In [ ]:
citibikes_df_2.drop(list(big_2[big_2['birth year'] <= 1939]['birth year'].index),inplace=True)

### the youngest riders would need to be born in 2003, so we check for any riders claiming to be born after that

In [ ]:
citibikes_df['birth year'].max() #looks like no one is reporting to be younger

### Smooth '1969' birth year


In [ ]:
def change_birth_year(row,med_age_df):
    '''
    This function will be applied to each row, where ever there is a 1969 we will replace it with the birth year 
    from median_birthyear_no_69 which matches with our stop neighborhood
    '''
    if row['birth year'] == 1969:
        year = med_age_df[med_age_df['end_nhbr']== row['end_nhbr']]['birth year']
        return year.item()
    else:
        return row['birth year']

Create dataframes of the median birth year by stop neighborhood

In [ ]:
median_birthyear = pd.DataFrame(citibikes_df.groupby('end_nhbr')['birth year'].median())
median_birthyear.reset_index(inplace=True) 
# median_birthyear.head()

In [ ]:
median_birthyear_2 = pd.DataFrame(citibikes_df_2.groupby('end_nhbr')['birth year'].median())
median_birthyear_2.reset_index(inplace=True) 
# median_birthyear.head()

In [ ]:
citibikes_df['update_birth_yr_2'] = citibikes_df.apply(lambda row: change_birth_year(row,median_birthyear),axis=1)
citibikes_df_2['update_birth_yr_2'] = citibikes_df_2.apply(lambda row: change_birth_year(row,median_birthyear_2),axis=1)

Create a displot of the fixed birth year column:

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(10,7))
sns.distplot(citibikes_df['birth year'])
plt.title('Distribution of Reported Ages w/o 1969')
plt.savefig('fixed_age_dist.png')

### Bin Age into Categories

### Label Encode for Morning versus Evening. 

To better fit a model, using a split day rather than by hour might aide in determining a destination. The idea is that many nieghborhoods are destinations for commuters going to work in the morning and alternatively heading home in the evening. 

### Remove redundant rows - 

Now that our primary categories are set we will remove features that are not needed or redundant. 

In [ ]:
citibikes_df = citibikes_df.drop(['tripduration', 'starttime', 'stoptime','start station name',
                'end station id', 'bikeid', 'start station latitude', 
                'start station longitude', 'end station latitude', 
                'end station longitude','end station name', 'start_date'], axis=1)

### Save to CSV file to be used in later models 

In [ ]:
#citibikes_df.to_csv('citibike_2018.csv', index=False)

### View Correlation between varibles. 

In [ ]:
sns.pairplot(df_small)

### View the Volume of stop Neighborhoods.

In [ ]:
plt.figure(figsize=(12,10))
sns.countplot(df_small.stop_nhbr)
plt.xticks(rotation=90)